# Iterative Approach for Unet Training 

------

## Overview



The purpose of this learn.ipynb notebook is to investigate whether an image can exhibit a preference for being segmented more effectively using a UNet model trained on polar or cartesian-dominant images.




-----


## File Structure
```
data
└── endoscopic
    ├── cartesian
    │   ├── image
    │   └── label
    └── polar
        ├── image
        └── label
```

Inside of each end folder there are 956 images, named as `0.tif` to `955.tif`
and I believe, for now, the naming of the images are one to one correctly matched, meaning the ``/data/endoscopic/**cartesian**/image/0.tif`` is transformed from `/data/endoscopic/**polar**/image/0.tif`

Instead of putting a seperate set of images aside to be test set, we chose to use k-fold cross validation.

In [ ]:
# Import from files and libraries

In [2]:
from defines import *
from model import *
from data import *
import sys

from numpy import loadtxt
from keras.models import load_model
from PIL import Image

import glob
import os
import math
import shutil

In [3]:
#run this if your computer has a cuda visible device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
print(PARAM_SYSTEM_TIME)

20230906_140606


In [5]:
# Test code for correct file structure setup

In [6]:
# Visualize folder tree in current directory
os.system("tree -d")

.
├── data
│   └── endoscopic
│       ├── cartesian
│       │   ├── image
│       │   └── label
│       └── polar
│           ├── image
│           └── label
├── dependencies
├── legacy
└── __pycache__

11 directories


0

In [7]:
#count of files in data directories
os.system("ls " + os.path.join(PARAM_PATH_CARTE,PARAM_IMG_FOLDER) + " | wc -l")
os.system("ls " + os.path.join(PARAM_PATH_CARTE,PARAM_MSK_FOLDER) + " | wc -l")
os.system("ls " + os.path.join(PARAM_PATH_POLAR,PARAM_IMG_FOLDER) + " | wc -l")
os.system("ls " + os.path.join(PARAM_PATH_POLAR,PARAM_MSK_FOLDER) + " | wc -l")


956
956
956
956


0

**Expected Output: 
956
956
956
956
**

-----

## Model training test

This part is just to see if we can train a model using the current configuration.

In [ ]:
# Setting Superparameters (temporary) for a test run of model training test

In [24]:
batch_size = 2
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
temp_save_dir = os.path.join(os.getcwd(),'temp')
test_gene = trainGenerator(batch_size, PARAM_PATH_CARTE, PARAM_IMG_FOLDER, PARAM_MSK_FOLDER, data_gen_args)
test_model = unet() 
test_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_168 (Conv2D)            (None, 256, 256, 64  1792        ['input_8[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_169 (Conv2D)            (None, 256, 256, 64  36928       ['conv2d_168[0][0]']             
                                )                                                           

                                8)                                                                
                                                                                                  
 conv2d_186 (Conv2D)            (None, 128, 128, 12  147584      ['conv2d_185[0][0]']             
                                8)                                                                
                                                                                                  
 up_sampling2d_31 (UpSampling2D  (None, 256, 256, 12  0          ['conv2d_186[0][0]']             
 )                              8)                                                                
                                                                                                  
 conv2d_187 (Conv2D)            (None, 256, 256, 64  32832       ['up_sampling2d_31[0][0]']       
                                )                                                                 
          

### Expected output
<details>
    <summary><b><font color="green">Click here to expand</font></b></summary>
    <code>
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
==================================================================================================
 input_6 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_120 (Conv2D)            (None, 256, 256, 64  1792        ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_121 (Conv2D)            (None, 256, 256, 64  36928       ['conv2d_120[0][0]']             
                                )                                                                 
                                                                                                  
 max_pooling2d_20 (MaxPooling2D  (None, 128, 128, 64  0          ['conv2d_121[0][0]']             
 )                              )                                                                 
                                                                                                  
 conv2d_122 (Conv2D)            (None, 128, 128, 12  73856       ['max_pooling2d_20[0][0]']       
                                8)                                                                
                                                                                                  
 conv2d_123 (Conv2D)            (None, 128, 128, 12  147584      ['conv2d_122[0][0]']             
                                8)                                                                
                                                                                                  
 max_pooling2d_21 (MaxPooling2D  (None, 64, 64, 128)  0          ['conv2d_123[0][0]']             
 )                                                                                                
                                                                                                  
 conv2d_124 (Conv2D)            (None, 64, 64, 256)  295168      ['max_pooling2d_21[0][0]']       
                                                                                                  
 conv2d_125 (Conv2D)            (None, 64, 64, 256)  590080      ['conv2d_124[0][0]']             
                                                                                                  
 max_pooling2d_22 (MaxPooling2D  (None, 32, 32, 256)  0          ['conv2d_125[0][0]']             
 )                                                                                                
                                                                                                  
 conv2d_126 (Conv2D)            (None, 32, 32, 512)  1180160     ['max_pooling2d_22[0][0]']       
                                                                                                  
 conv2d_127 (Conv2D)            (None, 32, 32, 512)  2359808     ['conv2d_126[0][0]']             
                                                                                                  
 dropout_10 (Dropout)           (None, 32, 32, 512)  0           ['conv2d_127[0][0]']             
                                                                                                  
 max_pooling2d_23 (MaxPooling2D  (None, 16, 16, 512)  0          ['dropout_10[0][0]']             
 )                                                                                                
                                                                                                  
 conv2d_128 (Conv2D)            (None, 16, 16, 1024  4719616     ['max_pooling2d_23[0][0]']       
                                )                                                                 
                                                                                                  
 conv2d_129 (Conv2D)            (None, 16, 16, 1024  9438208     ['conv2d_128[0][0]']             
                                )                                                                 
                                                                                                  
 dropout_11 (Dropout)           (None, 16, 16, 1024  0           ['conv2d_129[0][0]']             
                                )                                                                 
                                                                                                  
 up_sampling2d_20 (UpSampling2D  (None, 32, 32, 1024  0          ['dropout_11[0][0]']             
 )                              )                                                                 
                                                                                                  
 conv2d_130 (Conv2D)            (None, 32, 32, 512)  2097664     ['up_sampling2d_20[0][0]']       
                                                                                                  
 concatenate_20 (Concatenate)   (None, 32, 32, 1024  0           ['dropout_10[0][0]',             
                                )                                 'conv2d_130[0][0]']             
                                                                                                  
 conv2d_131 (Conv2D)            (None, 32, 32, 512)  4719104     ['concatenate_20[0][0]']         
                                                                                                  
 conv2d_132 (Conv2D)            (None, 32, 32, 512)  2359808     ['conv2d_131[0][0]']             
                                                                                                  
 up_sampling2d_21 (UpSampling2D  (None, 64, 64, 512)  0          ['conv2d_132[0][0]']             
 )                                                                                                
                                                                                                  
 conv2d_133 (Conv2D)            (None, 64, 64, 256)  524544      ['up_sampling2d_21[0][0]']       
                                                                                                  
 concatenate_21 (Concatenate)   (None, 64, 64, 512)  0           ['conv2d_125[0][0]',             
                                                                  'conv2d_133[0][0]']             
                                                                                                  
 conv2d_134 (Conv2D)            (None, 64, 64, 256)  1179904     ['concatenate_21[0][0]']         
                                                                                                  
 conv2d_135 (Conv2D)            (None, 64, 64, 256)  590080      ['conv2d_134[0][0]']             
                                                                                                  
 up_sampling2d_22 (UpSampling2D  (None, 128, 128, 25  0          ['conv2d_135[0][0]']             
 )                              6)                                                                
                                                                                                  
 conv2d_136 (Conv2D)            (None, 128, 128, 12  131200      ['up_sampling2d_22[0][0]']       
                                8)                                                                
                                                                                                  
 concatenate_22 (Concatenate)   (None, 128, 128, 25  0           ['conv2d_123[0][0]',             
                                6)                                'conv2d_136[0][0]']             
                                                                                                  
 conv2d_137 (Conv2D)            (None, 128, 128, 12  295040      ['concatenate_22[0][0]']         
                                8)                                                                
                                                                                                  
 conv2d_138 (Conv2D)            (None, 128, 128, 12  147584      ['conv2d_137[0][0]']             
                                8)                                                                
                                                                                                  
 up_sampling2d_23 (UpSampling2D  (None, 256, 256, 12  0          ['conv2d_138[0][0]']             
 )                              8)                                                                
                                                                                                  
 conv2d_139 (Conv2D)            (None, 256, 256, 64  32832       ['up_sampling2d_23[0][0]']       
                                )                                                                 
                                                                                                  
 concatenate_23 (Concatenate)   (None, 256, 256, 12  0           ['conv2d_121[0][0]',             
                                8)                                'conv2d_139[0][0]']             
                                                                                                  
 conv2d_140 (Conv2D)            (None, 256, 256, 64  73792       ['concatenate_23[0][0]']         
                                )                                                                 
                                                                                                  
 conv2d_141 (Conv2D)            (None, 256, 256, 64  36928       ['conv2d_140[0][0]']             
                                )                                                                 
                                                                                                  
 conv2d_142 (Conv2D)            (None, 256, 256, 9)  5193        ['conv2d_141[0][0]']             
                                                                                                  
 conv2d_143 (Conv2D)            (None, 256, 256, 3)  30          ['conv2d_142[0][0]']             
                                                                                                  
==================================================================================================
Total params: 31,036,903
Trainable params: 31,036,903
Non-trainable params: 0
</code>
</details>    

In [28]:
test_run = test_model.fit(test_gene, verbose = 2, steps_per_epoch = 50, epochs = 12, callbacks = [early_stop])#TODO: I am working on this line now

Epoch 1/12
50/50 - 7s - loss: 0.1725 - accuracy: 0.0432 - 7s/epoch - 142ms/step
Epoch 2/12
50/50 - 7s - loss: 0.1966 - accuracy: 0.1609 - 7s/epoch - 142ms/step
Epoch 3/12
50/50 - 7s - loss: 0.1786 - accuracy: 0.3772 - 7s/epoch - 140ms/step
Epoch 4/12
50/50 - 7s - loss: 0.1551 - accuracy: 0.4692 - 7s/epoch - 142ms/step
Epoch 5/12
50/50 - 7s - loss: 0.1575 - accuracy: 0.4283 - 7s/epoch - 141ms/step
Epoch 6/12
50/50 - 7s - loss: 0.1670 - accuracy: 0.3481 - 7s/epoch - 141ms/step
Epoch 7/12
50/50 - 7s - loss: 0.1423 - accuracy: 0.2396 - 7s/epoch - 141ms/step
Epoch 8/12
50/50 - 7s - loss: 0.1480 - accuracy: 0.4278 - 7s/epoch - 141ms/step
Epoch 9/12
50/50 - 7s - loss: 0.1331 - accuracy: 0.4519 - 7s/epoch - 142ms/step
Epoch 10/12
50/50 - 7s - loss: 0.1445 - accuracy: 0.3882 - 7s/epoch - 140ms/step
Epoch 11/12
50/50 - 7s - loss: 0.1481 - accuracy: 0.3463 - 7s/epoch - 142ms/step
Epoch 12/12
50/50 - 7s - loss: 0.1560 - accuracy: 0.3203 - 7s/epoch - 141ms/step


In [30]:
PARAM_PATH_TEST = './test'
image_name = '29.tif'
img = io.imread(os.path.join(PARAM_PATH_TEST,image_name),as_gray = False)
img = trans.resize(img,[256,256])
img = np.reshape(img,(1,)+img.shape)

results = test_model.predict(img,1,verbose=1)
#saveResult(Path,results)
img = results[0,:,:]
print(results.shape)
io.imsave(os.path.join(PARAM_PATH_TEST,"result.png"),img)

1/1 [==============================] - 0s 16ms/step


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


(1, 256, 256, 3)


In [ ]:
import numpy as np

#load pny file
img_score = np.load(PARAM_PATH_SCORES)
#sort scores in descending order and store index
sorted_score = np.flip(np.argsort(img_score))

#fetch top polar dominant and non-polar dominant image
num_polar = round(len(sorted_score)/2)
num_cartesian = len(sorted_score) - num_polar
dfPolar = sorted_score.head(num_polar)
dfCartesian = sorted_score.tail(num_cartesian)

In [ ]:
#K fold Validation (obtain training & testing sets)
from sklearn.model_selection import KFold

kfold = KFold(n_splits=PARAM_SPLIT_NUM)
for train_index,test_index in kfold.split(dfPolar):
    polar_train,polar_test=dfPolar[train_index],dfPolar[test_index]
    cartesian_train,cartesian_test=dfCartesian[train_index],dfCartesian[test_index]
    print(polar_train,polar_test)
    print(cartesian_train,cartesian_test)

-----

# Legacy Code Below
### written by Wenfan

In [ ]:
from defines import *
from model import *
from data import *
import sys

from numpy import loadtxt
from keras.models import load_model
from PIL import Image

import glob
import os 
import math
import shutil

In [ ]:
path = 'data_jupyter'# main folder path

D = 'all_images'
L = 'all_labels'

D_P_im = 'polar_im'
D_C_im = 'car_im'
D_P = 'polar_l'
D_C = 'car_l'

prev_number_of_D_P = -1
prev_number_of_D_C = -1

diff = math.inf

model_P = unet(PARAM_BETA1[PARAM_BETA_TEST_NUM], PARAM_BETA2[PARAM_BETA_TEST_NUM])
model_C = unet(PARAM_BETA1[PARAM_BETA_TEST_NUM], PARAM_BETA2[PARAM_BETA_TEST_NUM])

In [ ]:
#fill in the weights here
model_P.load_weights() 
model_C.load_weights() 

In [ ]:
count = 0

#change file name of D, L
for count, filename in enumerate(os.listdir(path+'/'+D)): 
    dst = str(count) + ".png"
    src = path+'/'+D+'/'+filename
    dst = path+'/'+D+'/'+dst

    os.rename(src, dst) 
    
for count, filename in enumerate(os.listdir(path+'/'+L)): 
    dst = str(count) + ".tif"
    src = path+'/'+L+'/'+filename
    dst = path+'/'+L+'/'+dst

    os.rename(src, dst) 

In [ ]:
def dscore(im, im_name):
    label = Image.open(os.path.join(L, im_name))
    
    pixelIm = im.load()
    pixelLabel = label.load()
    
    upper = 0
    lower = im.size[0] * im.size[1]
    
    for i in range(im.size[0]):
        for j in range(im.size[1]):
            if pixelIm[i,j] == pixelLabel[i,j]:
                upper = upper + 1
            
    upper = upper * 2
    
    return upper / lower
                

In [ ]:
while (diff > 0): 
    #load model
    #if count > 0:
    #    model_P.load_weights('model_P_weights_' + (count - 1))
    #    model_C.load_weights('model_C_weights_' + (count - 1))
    
    count = count + 1
    
    testGene_P, testGene_C = testGenerator(path,D,L)
    
    #perdict
    results_P = model_P.predict(testGene_P, PARAM_N_TESTS, verbose=1)
    results_C = model_C.predict(testGene_C, PARAM_N_TESTS, verbose=1)

    np.save(PARAM_PATH_TEST_NPY_P, results_P)
    saveResult(PARAM_PATH_TEST_RESULTS_P, results_P)
    np.save(PARAM_PATH_TEST_NPY_C, results_C)
    saveResult(PARAM_PATH_TEST_RESULTS_C, results_C)
    
    mergeIm(path, D, L, PARAM_PATH_TEST_RESULTS_P, path+'/'+D_P)
    mergeIm(path, D, L, PARAM_PATH_TEST_RESULTS_C, path+'/'+D_C)
    
    #change file name of D_P, D_C
    for count, filename in enumerate(os.listdir(path+'/'+D_P)): 
        dst = str(count) + ".tif"
        src = path+'/'+D_P+'/'+filenaLme
        dst = path+'/'+D_P+'/'+dst

        os.rename(src, dst) 
  
    for count, filename in enumerate(os.listdir(path+'/'+D_C)): 
        dst = str(count) + ".tif"
        src = path+'/'+D_C+'/'+filename
        dst = path+'/'+D_C+'/'+dst

        os.rename(src, dst) 
    
    #find the better one (based on L) and modify D_P, D_P_im, D_C, D_C_im
    for file in os.listdir(D):
        im_P = Image.open(path+'/'+D_P+'/'+file)
        im_C = Image.open(path+'/'+D_C+'/'+file)
        
        if dscore(im_P, file) > dscore(im_C, file):
            shutil.copyfile(path+'/'+D+'/'+file, path+'/'+D_P_im+'/'+file)
            os.remove(path+'/'+D_C+'/'+file)
        else:
            shutil.copyfile(path+'/'+D+'/'+file, path+'/'+D_C_im+'/'+file)
            os.remove(path+'/'+D_P+'/'+file)
            
    number_of_D_P = len(glob.glob(D_P))
    number_of_D_C = len(glob.glob(D_C))

    # file numbers difference
    diff = Math.abs(prev_number_of_D_P - number_of_D_P + prev_number_of_D_C - number_of_D_C) / 2
    
    prev_number_of_D_P = number_of_D_P
    prev_number_of_D_C = number_of_D_P
    
    #train model_P only on D_P, model_C only on D_C
    myGene_P = trainGenerator(PARAM_BATCHES, 
                            path, 
                            D_P, 
                            D_P_im, 
                            PARAM_DATA_ARGS, 
                            save_to_dir = PARAM_AUG_FOLDER_P)
    
    myGene_C = trainGenerator(PARAM_BATCHES, 
                            path, 
                            D_C, 
                            D_C_im, 
                            PARAM_DATA_ARGS, 
                            save_to_dir = PARAM_AUG_FOLDER_C)
    
    model_checkpoint_P = ModelCheckpoint( PARAM_SAVED_MODEL, 
                                         monitor = PARAM_METRICS, 
                                         verbose = 1, 
                                         save_best_only = PARAM_SAVE_BEST_ONLY)
    
    model_P.fit_generator(myGene_P,
                        steps_per_epoch = PARAM_EPOCH_STEPS,
                        epochs = PARAM_N_EPOCHS,
                        callbacks = [model_checkpoint])
    model_C.fit_generator(myGene_C,
                        steps_per_epoch = PARAM_EPOCH_STEPS,
                        epochs = PARAM_N_EPOCHS,
                        callbacks = [model_checkpoint])
    
    model_P.save_weights('model_P_weights_' + count)
    model_C.save_weights('model_C_weights_' + count)
    
    shutil.rmtree(path+'/'+D_P)
    shutil.rmtree(path+'/'+D_C)
    shutil.rmtree(path+'/'+D_P_im)
    shutil.rmtree(path+'/'+D_C_im)
    

save_model(model_P, 'model_P.h5')
save_model(model_C, 'model_C.h5')
    